# H2O Avocado

#### Goal :

In [ ]:
- Create a ML model using Auto-sklearn for the Avocado dataset
- Get RMSE over the predictions of these model

#### Imports

In [1]:
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.automl import H2OAutoML
from time import process_time

Initialtisation of an internal server used by H2O

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_152-release"; OpenJDK Runtime Environment (build 1.8.0_152-release-1056-b12); OpenJDK 64-Bit Server VM (build 25.152-b12, mixed mode)
  Starting server from /opt/conda/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp9w15t7cu
  JVM stdout: /tmp/tmp9w15t7cu/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp9w15t7cu/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.4
H2O_cluster_version_age:,20 days
H2O_cluster_name:,H2O_from_python_unknownUser_5slrhu
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,232.5 Mb
H2O_cluster_total_cores:,1
H2O_cluster_allowed_cores:,1
H2O_cluster_status:,"accepting new members, healthy"


importing our dataset and defining each column as "factor" or "predictor"

In [15]:
df = h2o.upload_file('../../Data/avocado_price/processed/train.csv')
response = "C1"
df[response] = df[response].asfactor()
predictors=[]
for col in df.columns:
    if col != response:
        predictors.append(col)

Parse progress: |█████████████████████████████████████████████████████████| 100%


C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15
,nan,4046,4225,4770,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
161,2.57698e+06,1.07153e+06,1.24107e+06,876,263513,263513,0,0,2017,42,2,5,0,1
173,562710,4328,250616,69819,237946,224187,13760,0,2015,37,10,18,0,1
152,7.46494e+07,7.1155e+06,4.32957e+07,2.85052e+06,2.13877e+07,2.10316e+07,228585,127500,2016,2,8,7,1,0
87,8.05438e+07,1.17059e+07,4.35492e+07,1.36935e+06,2.39194e+07,4.16884e+06,1.97488e+07,1763,2015,12,10,18,1,0
113,4.07205e+07,3.1008e+07,3.79532e+06,86746,5.83038e+06,4.60236e+06,1.22803e+06,0,2015,1,7,12,1,0
160,4.18469e+06,111730,837118,11539,3.22431e+06,3.22431e+06,0,0,2017,2,7,30,0,1
70,9.23022e+07,9.71363e+06,2.5673e+07,1.17622e+06,5.57393e+07,5.54443e+07,165304,129717,2016,36,12,11,1,0
224,952079,78153,327490,1927,544509,541870,2639,0,2016,32,9,25,0,1
115,2.00708e+06,839102,64417,0,1.10356e+06,823837,279723,0,2016,11,4,10,0,1


Impoerting our train and test dataset

In [16]:
train = h2o.upload_file('../../Data/avocado_price/processed/train.csv')
valid = h2o.upload_file('../../Data/avocado_price/processed/test.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15
nan,nan,4046,4225,4770,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
161,2.57698e+06,1.07153e+06,1.24107e+06,876,263513,263513,0,0,2017,42,2,5,0,1
173,562710,4328,250616,69819,237946,224187,13760,0,2015,37,10,18,0,1
152,7.46494e+07,7.1155e+06,4.32957e+07,2.85052e+06,2.13877e+07,2.10316e+07,228585,127500,2016,2,8,7,1,0
87,8.05438e+07,1.17059e+07,4.35492e+07,1.36935e+06,2.39194e+07,4.16884e+06,1.97488e+07,1763,2015,12,10,18,1,0
113,4.07205e+07,3.1008e+07,3.79532e+06,86746,5.83038e+06,4.60236e+06,1.22803e+06,0,2015,1,7,12,1,0
160,4.18469e+06,111730,837118,11539,3.22431e+06,3.22431e+06,0,0,2017,2,7,30,0,1
70,9.23022e+07,9.71363e+06,2.5673e+07,1.17622e+06,5.57393e+07,5.54443e+07,165304,129717,2016,36,12,11,1,0
224,952079,78153,327490,1927,544509,541870,2639,0,2016,32,9,25,0,1
115,2.00708e+06,839102,64417,0,1.10356e+06,823837,279723,0,2016,11,4,10,0,1


Creating our model

In [17]:
avocado_gbm = H2OGradientBoostingEstimator()

t1_start = process_time()
avocado_gbm.train(x = predictors,
               y = response,
               training_frame = train,
               validation_frame = valid)
t1_stop = process_time()


print("Elapsed time in seconds : ",t1_stop-t1_start)

gbm Model Build progress: |███████████████████████████████████████████████| 100%
Elapsed time in seconds :  0.22595822900000018


Getting our informations such as :
- Most important features
- RMSE

In [18]:
print(avocado_gbm)

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1592829396011_1


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,22888.0,5.0,5.0,5.0,28.0,32.0,31.76




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 304.76133581227435
RMSE: 17.457414923529612
MAE: 12.945316618600025
RMSLE: 0.12028877194119383
Mean Residual Deviance: 304.76133581227435

ModelMetricsRegression: gbm
** Reported on validation data. **

MSE: 320.7537228290488
RMSE: 17.909598622779036
MAE: 13.19257808011393
RMSLE: 0.12163843074418032
Mean Residual Deviance: 320.7537228290488

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
0,,2020-06-22 12:45:53,0.079 sec,0.0,40.258009,32.394034,1620.707257,40.300070,32.491648,1624.095629
1,,2020-06-22 12:45:55,1.778 sec,1.0,37.948379,30.303434,1440.079486,37.979833,30.390972,1442.467703
2,,2020-06-22 12:45:56,2.210 sec,2.0,35.959164,28.521763,1293.061498,36.004381,28.614287,1296.315483
3,,2020-06-22 12:45:56,2.497 sec,3.0,34.252066,27.002218,1173.204012,34.299415,27.077477,1176.449893
4,,2020-06-22 12:45:56,2.802 sec,4.0,32.722586,25.640181,1070.767647,32.760954,25.692737,1073.280113
5,,2020-06-22 12:45:56,3.044 sec,5.0,31.397489,24.476458,985.802327,31.437000,24.510104,988.284996
6,,2020-06-22 12:45:57,3.166 sec,6.0,30.238006,23.465059,914.336978,30.282477,23.483997,917.028419
7,,2020-06-22 12:45:57,3.280 sec,7.0,29.263099,22.619207,856.328952,29.322755,22.620759,859.823937
8,,2020-06-22 12:45:57,3.390 sec,8.0,28.392932,21.873301,806.158564,28.461882,21.857017,810.078746
9,,2020-06-22 12:45:57,3.507 sec,9.0,27.562460,21.172833,759.689218,27.662131,21.177175,765.193473



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,C14,3.801056e+07,1.000000,0.448928
1,C12,8.367678e+06,0.220141,0.098827
2,C10,7.129487e+06,0.187566,0.084204
3,C8,6.602849e+06,0.173711,0.077984
4,C3,6.151405e+06,0.161834,0.072652
5,C11,5.964706e+06,0.156922,0.070447
6,C4,4.791912e+06,0.126068,0.056595
7,C6,3.231093e+06,0.085005,0.038161
8,C7,1.607790e+06,0.042298,0.018989
9,C2,1.258340e+06,0.033105,0.014862
